In [1]:
import requests
import os 
import snowflake.connector
import time
import json as js
import flask

/Users/raph/Documents/rapmac-repo/.virtualenvs/scalability/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: dlopen(/Users/raph/Documents/rapmac-repo/.virtualenvs/scalability/lib/python3.9/site-packages/snowflake/connector/arrow_iterator.cpython-39-darwin.so, 0x0002): symbol not found in flat namespace '__ZN5arrow17Decimal128BuilderC1ERKNSt3__110shared_ptrINS_8DataTypeEEEPNS_10MemoryPoolE'


In [2]:
def getDomain(url):
    url = url.replace("https://www.", "").replace("http://www.", "").replace("www.","")
    return url

def querySFCompany(domain, user, password, account):
    DATABASE='dbt_db'
    SCHEMA='models_final'
    TABLE = 'final_companies_flatten'
    WAREHOUSE = 'COMPUTE_WH'
    ctx = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA
    )
    cs = ctx.cursor()
    query = "select count(*) from "+DATABASE+"."+SCHEMA+"."+TABLE+" where COMPANY_DOMAIN LIKE '" +domain+"'"
    print(query)
    cs.execute(query)
    query_id = cs.sfqid
    while ctx.is_still_running(ctx.get_query_status(query_id)):
        time.sleep(1)
    cs.get_results_from_sfqid(query_id)
    results = cs.fetchall()
    print(f'{results}')
    if results[0][0]!=0:
        query = "select * from "+DATABASE+"."+SCHEMA+"."+TABLE+" where COMPANY_DOMAIN LIKE '" +domain+"'"
        cs.execute(query)
        query_id = cs.sfqid
        while ctx.is_still_running(ctx.get_query_status(query_id)):
            time.sleep(1)
        cs.get_results_from_sfqid(query_id)
        results = cs.fetchall()
        print('type result' + str(type(results[0])))
        print(results[0][1])
        ctx.close()
        return 'True', results
    else:
        ctx.close()
        results=''
        return 'False', results

def getResultsFromCompanySF(results:list):
    results_cleaned = {'company_linkedin_universal_name':results[2],
                       'company_linkedin_id' : results[1],
                       'company_founded_year' :  results[7],
                       'company_country_code' : results[8],
                       'company_employees_count' : results[12],
                       'company_linkedin_industry' : results[13],
                       'company_captain_data_employee_range' : results[52],
                       'company_captain_data_extracted_at' : str(results[41])}
    return results_cleaned
    
def scrappingURL1(domain, api_key):
    return "https://app.scrapingbee.com/api/v1/?api_key="+api_key+"&url=http://www."+domain+"&render_js=false&extract_rules={%22links%22:%20{%22selector%22:%20%22//a[contains(@href,%27linkedin.com/company%27)]%22,%20%22type%22:%20%22list%22,%20%22output%22:%20%22@href%22}}"

def scrappingURL2(domain,api_key):
    return "https://app.scrapingbee.com/api/v1/?api_key="+api_key+"&url=https%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dsite%253Alinkedin.com%252Fcompany%2B"+domain+"&extract_rules=%7B%22first_link%22%3A%7B%22selector%22%3A%22%23search%20a%22%2C%22output%22%3A%22%40href%22%7D%7D&custom_google=True"

def hubspot_checker(domain, user, password, account, customer):
    DATABASE='airbyte_db'
    SCHEMA='crm_hubspot_'+customer
    TABLE = 'companies'
    WAREHOUSE = 'COMPUTE_WH'
    ctx = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA
    )
    cs = ctx.cursor()
    query = "select count(*) from "+DATABASE+"."+SCHEMA+"."+TABLE+" where PROPERTIES LIKE '%" +domain+"%'"
    cs.execute(query)
    query_id = cs.sfqid
    while ctx.is_still_running(ctx.get_query_status(query_id)):
        time.sleep(1)
    cs.get_results_from_sfqid(query_id)
    results = cs.fetchall()
    print(f'{results}')
    ctx.close()
    if results[0][0]!=0:
        #display = 'La company est présente dans le CRM Hubspot de '+customer
        #return display
        return True
    else:
        #display = 'La company n\'est pas présente dans le CRM Hubspot de '+customer 
        #return display
        return False

def valid_accounts(captain_data_api_key):
    auth = "x-api-key " + captain_data_api_key
    url = "https://api.captaindata.co/v3/integrations/linkedin/accounts?page=0"

    headers = {
        "Content-Type": "application/json",
        "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
        "Authorization": auth
    }

    response = requests.get(url, headers=headers)
    result= json.loads(response.text)
    #print(result)
    uid = ""
    for i in range(len(result['accounts'])):
        if result['accounts'][i]['is_valid'] == True:
            uid = result['accounts'][i]['uid']
            print(result['integration'])
            print(result['accounts'][i]['name'])
    #return 'Aucun compte LinkedIn n\'est actuellement valide sur Captain Data'
    if uid == "":
        return 'Aucun compte LinkedIn n\'est actuellement valide sur Captain Data'
    else:
        return uid
    #print(result['accounts'][0]['is_valid']) 

def launch_job(uid, captain_data_api_key, linkedin_url):
    url = "https://api.captaindata.co/v3/workflows/b7aaec34-6e36-40b3-9f1f-376276d07af1/schedule"
    auth = "x-api-key " + captain_data_api_key

    headers = {
        "Content-Type": "application/json",
        "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
        "Authorization": auth
    }

    body = {
        "steps": [
            {
                "accounts": [
                    uid
                ],
                "parameters": {
                    "extractOpenPositions": False
                },
                "output_column": None,
                "step_uid": "964f6c7c-9c73-4aff-a6e9-790f0c836565"
            }
        ],
        "inputs": [
            {
                "linkedin_company_url": linkedin_url,
                "url": linkedin_url
            }
        ],
        "job_name": uid
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))
    result= json.loads(response.text)
    #print(response.text)
    print(result['job_uid'])
    return result['job_uid']

def status_job(job_uid, captain_data_api_key):
    url = "https://api.captaindata.co/v3/jobs/"+job_uid+"/results"
    auth = "x-api-key " + captain_data_api_key
    headers = {
        "Content-Type": "application/json",
        "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
        "Authorization": auth
    }
    response = requests.get(url, headers=headers)
    result = json.loads(response.text)
    print(type(result))
    count = 1
    while result['items_count'] == 0:
        print('Essai '+str(count))
        time.sleep(5)
        response = requests.get(url, headers=headers)
        result = json.loads(response.text)
        count+=1
    print(result)
    return response.text

In [3]:
domain="getscalability.io"
user=os.environ['USER_SNOWFLAKE']
print(user)
password=os.environ['PASSWORD_SNOWFLAKE']
print(password)
account=os.environ['ACCOUNT_SNOWFLAKE']
print(account)
scrapping_bee_api_key = 'MLINMYF4NZMFT3WA37AGIJ54AP10F86H8TDXSFNJSOIG3NRXMKXE04JSEWID422DT3SDOPEHPE2F3VKC'
captain_data_api_key = 'feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d'
hubspot_checker(domain,user,password,account,'simbel')

PythonAPI
ScalabilityPython!2023
rq37585.eu-west-1
[(1,)]


True

In [44]:
def company_enrichment_oneshot(company_website:str):
    user=os.getenv('USER_SNOWFLAKE')
    password=os.getenv('PASSWORD_SNOWFLAKE')
    account=os.getenv('ACCOUNT_SNOWFLAKE')
    scrapping_bee_api_key = os.getenv('SCRAPPING_BEE_API_KEY') 
    captain_data_api_key = os.getenv('CAPTAIN_DATA_API_KEY') 
    domain = getDomain(company_website)
    print(domain)
    querySF=querySFCompany(domain,user,password,account)
    print('querySF result '+querySF[0])
    if querySF[0]=='False':
        x = requests.get(scrappingURL1(domain,scrapping_bee_api_key))
        print(x.text)
        result = json.loads(x.text)
        print(result)
        if len(result['links']) == 0:
            result = json.loads(requests.get(scrappingURL2(domain,scrapping_bee_api_key)).text)
            linkedin_url = result['first_link']
            print('linkedin url '+linkedin_url)
        elif len(result['links']) != 0 :
            linkedin_url = result['links'][0]
            print('linkedin url '+linkedin_url)
        uid = valid_accounts(captain_data_api_key)
        job_uid = launch_job(uid, captain_data_api_key, linkedin_url)
        result = status_job(job_uid,captain_data_api_key)
        inDB = 'False'
    
    else:
        print('Déjà présent dans la base Company')
        result = querySF[1][0]
        results = getResultsFromCompanySF(result)
        inDB = 'True'
    hubspot_join = hubspot_checker(domain, user, password, account, 'join')
    print(hubspot_join)
    hubspot_karmen = hubspot_checker(domain, user, password, account, 'karmen')
    print(hubspot_karmen)
    hubspot_open = hubspot_checker(domain, user, password, account, 'open')
    print(hubspot_open)
    hubspot_reecall = hubspot_checker(domain, user, password, account, 'reecall')
    print(hubspot_reecall)
    hubspot_simbel = hubspot_checker(domain, user, password, account, 'simbel')
    print(hubspot_simbel)
    results = {'inBB':inDB,
                'company_linkedin_universal_name':results['company_linkedin_universal_name'],
                'company_linkedin_id' : results['company_linkedin_id'],
                'company_founded_year' :  results['company_founded_year'],
                'company_country_code' : results['company_country_code'],
                'company_employees_count' : results['company_employees_count'],
                'company_linkedin_industry' : results['company_linkedin_industry'],
                'company_captain_data_employee_range' : results['company_captain_data_employee_range'],
                'company_captain_data_extracted_at' : results['company_captain_data_extracted_at'],
                'in_hubspot_join':hubspot_join, 
                'in_hubspot_karmen':hubspot_karmen, 
                'in_hubspot_open':hubspot_open, 
                'in_hubspot_reecall':hubspot_reecall, 
                'in_hubspot_simbel':hubspot_simbel}
    return results

In [43]:
company_enrichment_oneshot('getscalability.io')

getscalability.io
select count(*) from dbt_db.models_final.final_companies_flatten where COMPANY_DOMAIN LIKE 'getscalability.io'
[(1,)]
type result<class 'tuple'>
71230099
querySF result True
Déjà présent dans la base Company
('linkedin_universal_name:getscalability', '71230099', 'getscalability', 'Get Scalability', 'Scalability vous aide à améliorer vos process Sales & growth. ', 'getscalability.io', '', None, 'FR', '75013', 'Paris', None, 5, 'Internet', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, datetime.datetime(2023, 2, 21, 11, 18, 41, 514000), '71230099', 'getscalability', None, 'getscalability.io', 'Get Scalability', 'Internet', 'Scalability vous aide à améliorer vos process Sales & growth. ', 'Growth as a service. Put your Sales process on automation to accelerate and grow your funnels.', 'Privately Held', 5, '2 - 10', 241, None, None, 'Paris', 'FR', '75013', ''

{'inBB': 'True',
 'company_linkedin_universal_name': 'getscalability',
 'company_linkedin_id': '71230099',
 'company_founded_year': None,
 'company_country_code': 'FR',
 'company_employees_count': 5,
 'company_linkedin_industry': 'Internet',
 'company_captain_data_employee_range': '2 - 10',
 'company_captain_data_extracted_at': datetime.datetime(2023, 2, 21, 11, 18, 41, 514000),
 'in_hubspot_join': True,
 'in_hubspot_karmen': True,
 'in_hubspot_open': True,
 'in_hubspot_reecall': False,
 'in_hubspot_simbel': True}

In [74]:
url="http://www.reveal.com"
user=os.environ['USER_SNOWFLAKE']
password=os.environ['PASSWORD_SNOWFLAKE']
account=os.environ['ACCOUNT_SNOWFLAKE']
scrapping_bee_api_key = 'MLINMYF4NZMFT3WA37AGIJ54AP10F86H8TDXSFNJSOIG3NRXMKXE04JSEWID422DT3SDOPEHPE2F3VKC'
captain_data_api_key = 'feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d'
def workflow(url):
  domain = getDomain(url)
  print(domain)
  if querySFCompany(domain,user,password,account)==False:
    x = requests.get(scrappingURL1(getDomain(url),scrapping_bee_api_key))
    print(x.text)
    result = json.loads(x.text)
    print(result)
    if len(result['links']) == 0:
      result = json.loads(requests.get(scrappingURL2(getDomain(url),scrapping_bee_api_key)).text)
      linkedin_url = result['first_link']
      print('linkedin url '+linkedin_url)
    elif len(result['links']) != 0 :
      linkedin_url = result['links'][0]
      print('linkedin url '+linkedin_url)
    uid = valid_accounts(captain_data_api_key)
    job_uid = launch_job(uid, captain_data_api_key, linkedin_url)
    result = status_job(job_uid,captain_data_api_key)
    
  else:
    print('Déjà présent dans la base Company')
    result = 'Already in company DB'
  hubspot_join = hubspot_checker(domain, user, password, account, 'join')
  print(hubspot_join)
  hubspot_karmen = hubspot_checker(domain, user, password, account, 'karmen')
  print(hubspot_karmen)
  hubspot_open = hubspot_checker(domain, user, password, account, 'open')
  print(hubspot_open)
  hubspot_reecall = hubspot_checker(domain, user, password, account, 'reecall')
  print(hubspot_reecall)
  hubspot_simbel = hubspot_checker(domain, user, password, account, 'simbel')
  print(hubspot_simbel)
  results = {'result':result, 'in_hubspot_join':hubspot_join, 'in_hubspot_karmen':hubspot_karmen, 'in_hubspot_open':hubspot_open, 'in_hubspot_reecall':hubspot_reecall, 'in_hubspot_simbel':hubspot_simbel}
  return results
  


In [76]:
workflow('https://www.dataship.tech')

dataship.tech
select count(*) from dbt_db.models_final.final_companies_flatten where COMPANY_DOMAIN LIKE 'dataship.tech'
[(0,)]
{"links": []}
{'links': []}
linkedin url https://www.linkedin.com/company/dataships
{'uid': 'linkedin', 'title': 'LinkedIn'}
Adrien Tedjirian
{'uid': 'linkedin', 'title': 'LinkedIn'}
Victor Cariou
{'uid': 'linkedin', 'title': 'LinkedIn'}
Johann Lemena
{'uid': 'linkedin', 'title': 'LinkedIn'}
Arthur Morizur
{'uid': 'linkedin', 'title': 'LinkedIn'}
Victor Alexandrian
{'uid': 'linkedin', 'title': 'LinkedIn'}
Etienne Dubois
cb96fd4c-66e5-4e8a-9128-8b6e33de059e
<class 'dict'>
Essai 1
Essai 2
{'items_count': 1, 'pages': 1, 'limit': 1000, 'results': [{'updated_at': '2023-02-15T16:13:55.870000', 'company_name': 'Dataships', 'name': 'Dataships', 'specialties': ['Regulation', 'IT', 'Software', 'Compliance', 'GDPR', 'EU Law', 'Data Privacy', 'DSAR', 'DAR', 'SAR', 'Fitness', 'Financial Services', 'SaaS', 'Data Protection'], 'specialities': ['Regulation', 'IT', 'Software',

{'result': {'items_count': 1,
  'pages': 1,
  'limit': 1000,
  'results': [{'updated_at': '2023-02-15T16:13:55.870000',
    'company_name': 'Dataships',
    'name': 'Dataships',
    'specialties': ['Regulation',
     'IT',
     'Software',
     'Compliance',
     'GDPR',
     'EU Law',
     'Data Privacy',
     'DSAR',
     'DAR',
     'SAR',
     'Fitness',
     'Financial Services',
     'SaaS',
     'Data Protection'],
    'specialities': ['Regulation',
     'IT',
     'Software',
     'Compliance',
     'GDPR',
     'EU Law',
     'Data Privacy',
     'DSAR',
     'DAR',
     'SAR',
     'Fitness',
     'Financial Services',
     'SaaS',
     'Data Protection'],
    'tagline': 'We help online businesses grow their marketing audiences compliantly.',
    'linkedin_company_id': '11739039',
    'id': '11739039',
    'description': 'We specialize in data privacy compliance for SMEs and scaling start-ups. We created a SaaS product that automates compliance with data privacy laws such as 

In [4]:

query = querySFCompany(getDomain(url))
DATABASE='dbt_db'
SCHEMA='models.final'
ctx = snowflake.connector.connect(
    user=os.environ['USER_SNOWFLAKE'],
    password=os.environ['PASSWORD_SNOWFLAKE'],
    account=os.environ['ACCOUNT_SNOWFLAKE'],
    warehouse='COMPUTE_WH',
    database=DATABASE,
    schema=SCHEMA
    )
cs = ctx.cursor()
cs.execute(query)
query_id = cs.sfqid
while ctx.is_still_running(ctx.get_query_status(query_id)):
  time.sleep(1)
cs.get_results_from_sfqid(query_id)
results = cs.fetchall()
print(f'{results[0]}')
ctx.close()

TypeError: querySFCompany() missing 3 required positional arguments: 'user', 'password', and 'account'

In [45]:
x = requests.get(scrappingURL1(getDomain(url)))
print(x.text)
result = json.loads(x.text)

{"links": []}


In [46]:
print(result)
len(result)

{'links': []}


1

In [47]:
if len(result['links']) == 0:
    result = json.loads(requests.get(scrappingURL2(getDomain(url))).text)
    print(result['first_link'])
elif len(result['links']) != 0 :
    print(result['links'][0])

https://www.linkedin.com/company/reveal-group


In [48]:
def valid_accounts(captain_data_api_key):
    auth = "x-api-key " + captain_data_api_key
    url = "https://api.captaindata.co/v3/integrations/linkedin/accounts?page=0"

    headers = {
        "Content-Type": "application/json",
        "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
        "Authorization": auth
    }

    response = requests.get(url, headers=headers)
    result= json.loads(response.text)
    print(result)
    uid = ""
    for i in range(len(result['accounts'])):
        if result['accounts'][i]['is_valid'] == True:
            uid = result['accounts'][i]['uid']
            print(result['integration'])
            print(result['accounts'][i]['name'])
    #return 'Aucun compte LinkedIn n\'est actuellement valide sur Captain Data'
    if uid == "":
        return 'Aucun compte LinkedIn n\'est actuellement valide sur Captain Data'
    else:
        return uid
    #print(result['accounts'][0]['is_valid']) 

def launch_job(uid, captain_data_api_key, linkedin_url):
    url = "https://api.captaindata.co/v3/workflows/b7aaec34-6e36-40b3-9f1f-376276d07af1/schedule"
    auth = "x-api-key " + captain_data_api_key

    headers = {
        "Content-Type": "application/json",
        "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
        "Authorization": auth
    }

    body = {
        "steps": [
            {
                "accounts": [
                    uid
                ],
                "parameters": {
                    "extractOpenPositions": False
                },
                "output_column": None,
                "step_uid": "964f6c7c-9c73-4aff-a6e9-790f0c836565"
            }
        ],
        "inputs": [
            {
                "linkedin_company_url": linkedin_url,
                "url": linkedin_url
            }
        ],
        "job_name": uid
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))
    result= json.loads(response.text)
    print(result['uid'])

def status_job(job_uid, captain_data_api_key):
    job_uid = '41f38992-6064-46f1-a2c7-9df462a29bb5'
    url = "https://api.captaindata.co/v3/jobs/"+job_uid+"/results"
    auth = "x-api-key " + captain_data_api_key
    headers = {
        "Content-Type": "application/json",
        "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
        "Authorization": auth
    }
    response = requests.get(url, headers=headers)


{'integration': {'uid': 'linkedin', 'title': 'LinkedIn'}, 'accounts': [{'setup_at': '2023-02-06T16:23:23.114398', 'name': 'Victor Cariou', 'cookies': {'JSESSIONID': '"ajax:3377178882135116813"', 'li_a': None, 'li_at': 'AQEDASOu32kEADNMAAABhieJn4kAAAGGS5YjiVYAlbCNbWoTpTTM9B_qc6Et6ZdlPvM0tfkDsl3KeJavcaCxbALylTQCFgWf9qtZUC1m3JQ_jzX-vDSmOT9XZFFvh25P88sc8UrkzfJ24pOJ_FqO_rSw'}, 'via_extension': True, 'is_shared': True, 'level': 'Sales Navigator', 'is_valid': True, 'uid': 'eb0832aa-38d8-435f-8869-c4f53b00d747', 'application_permalink': 'linkedin', 'application_name': 'LinkedIn'}, {'setup_at': '2023-02-06T14:53:48.823359', 'name': 'Johann Lemena', 'cookies': {'JSESSIONID': '"ajax:3458321767039518452"', 'li_a': 'AQJ2PTEmc2FsZXNfY2lkPTQ5OTE2NDcwMiUzQSUzQTQ0NDIxMDAwN5l_w2kphKDfMQbZqRNsR_Cp7KB2', 'li_at': 'AQEDATLHmdQCGXV4AAABhhIru0QAAAGGNjg_RE4An5V_jUj8ccJEz2YvQzPj-Au-Q5f2JADCMXvsU4vXjM_pQRO4pIbvfx-E9KnYlJWLdtDNCU0iYbbeCZ5852iP7udccM-KsXx8WYp9zPFtCtLA0CpR'}, 'via_extension': True, 'is_shared': Tr

In [51]:
for i in range(len(result['accounts'])):
    if result['accounts'][i]['is_valid'] == True:
        uid = result['accounts'][i]['uid']
        print(result['integration'])
        print(result['accounts'][i]['name'])
#return 'Aucun compte LinkedIn n\'est actuellement valide sur Captain Data'
print('Aucun compte LinkedIn n\'est actuellement valide sur Captain Data')
#print(result['accounts'][0]['is_valid'])

eb0832aa-38d8-435f-8869-c4f53b00d747
{'uid': 'linkedin', 'title': 'LinkedIn'}
Victor Cariou
efbb4db4-c60b-4ff5-b6dd-25560894c8aa
{'uid': 'linkedin', 'title': 'LinkedIn'}
Johann Lemena
a4960e9d-52d4-4b6f-8fff-b606034bc8ad
{'uid': 'linkedin', 'title': 'LinkedIn'}
Arthur Morizur
077bf979-0b37-4ed6-a349-d3dee297aaeb
{'uid': 'linkedin', 'title': 'LinkedIn'}
Victor Alexandrian
3dad7d0a-25d5-4dfb-8e16-690006957061
{'uid': 'linkedin', 'title': 'LinkedIn'}
Etienne Dubois
d9bfcd2c-25d5-477c-a77f-33c2082c3ea1
{'uid': 'linkedin', 'title': 'LinkedIn'}
Benjamin Donteville
Aucun compte LinkedIn n'est actuellement valide sur Captain Data


In [14]:
print(result['accounts'][0])

{'setup_at': '2023-02-06T16:23:23.114398', 'name': 'Victor Cariou', 'cookies': {'JSESSIONID': '"ajax:3377178882135116813"', 'li_a': None, 'li_at': 'AQEDASOu32kEADNMAAABhieJn4kAAAGGS5YjiVYAlbCNbWoTpTTM9B_qc6Et6ZdlPvM0tfkDsl3KeJavcaCxbALylTQCFgWf9qtZUC1m3JQ_jzX-vDSmOT9XZFFvh25P88sc8UrkzfJ24pOJ_FqO_rSw'}, 'via_extension': True, 'is_shared': True, 'level': 'Sales Navigator', 'is_valid': True, 'uid': 'eb0832aa-38d8-435f-8869-c4f53b00d747', 'application_permalink': 'linkedin', 'application_name': 'LinkedIn'}


In [56]:
linkedinUrl = 'https://www.linkedin.com/company/reveal-group'
uid = "eb0832aa-38d8-435f-8869-c4f53b00d747"
url = "https://api.captaindata.co/v3/workflows/b7aaec34-6e36-40b3-9f1f-376276d07af1/schedule"

headers = {
    "Content-Type": "application/json",
    "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
    "Authorization": "x-api-key feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d"
}

body = {
    "steps": [
        {
            "accounts": [
                uid
            ],
            "parameters": {
                "extractOpenPositions": False
            },
            "output_column": None,
            "step_uid": "964f6c7c-9c73-4aff-a6e9-790f0c836565"
        }
    ],
    "inputs": [
        {
            "linkedin_company_url": linkedinUrl,
            "url": linkedinUrl
        }
    ],
    "job_name": uid
}

response = requests.post(url, headers=headers, data=json.dumps(body))
result= json.loads(response.text)
print(response.text)




{"template":null,"workflow":{"uid":"b7aaec34-6e36-40b3-9f1f-376276d07af1","permalink":"clay-extract-linkedin-company-profile3d301bbe-34fa-4514-9e30-a7cc2d446654","name":"CLAY - Extract LinkedIn Company Profile","created_at":"2023-02-03T14:37:00.586478","project_id":2774,"linked_templates":["8e315ab4-bce9-47ca-aaad-66defed46c61"],"template_uid":"8e315ab4-bce9-47ca-aaad-66defed46c61"},"message":"Bot successfully scheduled.","job_uid":"ac4b0a72-e543-4865-aac8-c552e177e4ba"}


In [60]:
print(result['job_uid'])

ac4b0a72-e543-4865-aac8-c552e177e4ba


In [48]:
url = "https://api.captaindata.co/v3/jobs/"+'ac4b0a72-e543-4865-aac8-c552e177e4ba'+"/results"

headers = {
    "Content-Type": "application/json",
    "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
    "Authorization": "x-api-key feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d"
}
response = requests.get(url, headers=headers)

In [55]:
js= json.loads(response.text)

In [46]:
job_uid = '834bb390-046a-40f8-ae12-94a872d0e893'
url = "https://api.captaindata.co/v3/jobs/"+job_uid+"/results"

headers = {
    "Content-Type": "application/json",
    "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
    "Authorization": "x-api-key feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d"
}
response = requests.get(url, headers=headers)

In [47]:
print(response.text)

NameError: name 'response' is not defined

In [67]:
print(js['results'])

[{'updated_at': '2023-02-09T10:21:56.148000', 'company_name': 'Reveal Group', 'name': 'Reveal Group', 'specialties': ['Operations Management', 'Workforce Management', 'Robotic Process Automation', 'Performance Management', 'Change Management', 'Enterprise Transformation', 'Business Process Management (BPM) Software', 'Business Process Automation Software', 'Intelligent Automation'], 'specialities': ['Operations Management', 'Workforce Management', 'Robotic Process Automation', 'Performance Management', 'Change Management', 'Enterprise Transformation', 'Business Process Management (BPM) Software', 'Business Process Automation Software', 'Intelligent Automation'], 'tagline': 'We help organizations #AutomateBetter. Proud UiPath USN Partner + only Blue Prism Double Platinum Partner. ', 'linkedin_company_id': '77340', 'id': '77340', 'description': "Reveal Group is your Intelligent Automation services company. We have been helping clients to Automate Better since 2005. Reveal Group is focuse

In [77]:
print(str(js['results'][0]['extracted_at']))

2023-02-09T10:21:56.151000


In [ ]:
universal_name = js['results'][0]['name']
linkedin_id = js['results'][0]['id']
linkedin_year_founded = js['results'][0]['year_founded']
linkedin_countr = js['results'][0]['country']
linkedin_employees_count = js['results'][0]['employees_count']
linkedin_industry = js['results'][0]['industry']
linkedin_employee_range = js['results'][0]['employees_range']
linkedin_extracted_at = str(js['results'][0]['extracted_at'])





